In [1]:
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import math
import cv2
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import glob
from matplotlib.image import imread

In [3]:
test_images = [plt.imread(image) for image in glob.glob('test_images/*.jpg')]

In [2]:
def calibrate_camera():
    """
        Calibrate camera from the provided calibration images
    """
    nx = 9
    ny = 6
    images = glob.glob('camera_cal/calibration*.jpg')
    
    print("Starting Calibrating Camera")
    # Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((nx*ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1, 2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Apply findChessboardCorners to the individual images
    for fname in images:    
        img = mpimg.imread(fname)
        image_shape = img.shape
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

    # Camera calibration, given object points, image points, and the shape of the grayscale image
    if (len(objpoints) > 0):
        # Camera successfully calibrated.
        print("Camera successfully calibrated.")
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, image_shape[:2], None, None)
    else:
        # Unable to calibrate the camera.
        print("Unable to calibrate the camera.")
        ret, mtx, dist, rvecs, tvecs = (None, None, None, None, None)

    return mtx, dist

mtx, dist = calibrate_camera()
    
def undistort(image):
    """
        Remove the distortion of an image 
    """
    return cv2.undistort(image, mtx, dist, None, mtx)

Starting Calibrating Camera
Camera successfully calibrated.


In [4]:
def show_images(images, cmap=None):
    """
      show list of images  
    """
    plt.figure(figsize=(40,40))    
    for i, image in enumerate(images):
        plt.subplot(math.ceil(len(images) / 2), 2, i+1)
        plt.imshow(image, cmap)
        plt.autoscale(tight=True)
    plt.show()
    
# show_images(test_images)

In [5]:
def apply(fun, images):
    """
        apply function to list of images
    """
    return list(map(fun, images))

In [6]:
def convert_to_hls(image):
    """
        Convert RGB image to HLS
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2HLS)

def convert_to_gray(image):
    """
        Convert RGB image to grayscale
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

def convert_to_rgp(image):
    """
        Convert RGB image to grayscale
    """
    return cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

gray_images = apply(convert_to_gray, test_images)
# show_images(gray_images, "gray")
hls_images = apply(convert_to_hls, test_images)
# show_images(hls_images)

In [8]:
def hls_color_threshold(image):
    """
        Extract yellow and white colors from image
    """
    # Convert the input image to HLS
    converted_image = convert_to_hls(image)
    
    # White color mask
    lower_threshold = np.uint8([0, 190, 0])
    upper_threshold = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(converted_image, lower_threshold, upper_threshold)
    
    # Yellow color mask
    lower_threshold = np.uint8([10, 0, 90])
    upper_threshold = np.uint8([50, 255, 255])
    yellow_mask = cv2.inRange(converted_image, lower_threshold, upper_threshold)
    
    # Combine white and yellow masks
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    masked_image = cv2.bitwise_and(image, image, mask = mask)
    
    return masked_image

hls_color_threshold_images = apply(hls_color_threshold, test_images)
# show_images(hls_color_threshold_images, "gray")

In [9]:
def apply_thresholds(image):
    """
        apply all the thresholds on image
    """
    hls = hls_color_threshold(image)
    gray = hls[:, :, 1]
    s_channel = hls[:, :, 2]
    
    sobel_binary = np.zeros(shape=gray.shape, dtype=bool)
    s_binary = sobel_binary
    combined_binary = s_binary.astype(np.float32)

    # Sobel Transform
    sobel_kernel=7 
    mag_thresh=(3, 255)
    s_thresh=(170, 255)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = 0 #cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    sobel_abs = np.abs(sobelx**2 + sobely**2)
    sobel_abs = np.uint8(255 * sobel_abs / np.max(sobel_abs))

    sobel_binary[(sobel_abs > mag_thresh[0]) & (sobel_abs <= mag_thresh[1])] = 1

    # Threshold color channel
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    # Combine the two binary thresholds

    combined_binary[(s_binary == 1) | (sobel_binary == 1)] = 1
    combined_binary = np.uint8(255 * combined_binary / np.max(combined_binary))
    
    return combined_binary

threshold_images = apply(apply_thresholds, test_images)
# show_images(threshold_images, "gray")

In [10]:
def extract_region_of_interest(image):
    x = int(image.shape[1])
    y = int(image.shape[0])
    shape = np.array([[0, y], [x, y], [int(0.55 * x), int(0.6 * y)], [int(0.45 * x), int(0.6 * y)]])

    # define a numpy array with the dimensions of image, but comprised of zeros
    mask = np.zeros_like(image)

    # Uses 3 channels or 1 channel for color depending on input image
    if len(image.shape) > 2:
        channel_count = image.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    # creates a polygon with the mask color
    cv2.fillPoly(mask, np.int32([shape]), ignore_mask_color)

    # returns the image only where the mask pixels are not zero
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

roi_images = apply(extract_region_of_interest, threshold_images)
# show_images(roi_images, "gray")

In [11]:
def warp(img):
    # Vertices extracted manually for performing a perspective transform
    bottom_left = [220,720]
    bottom_right = [1110, 720]
    top_left = [570, 470]
    top_right = [722, 470]

    source = np.float32([bottom_left,bottom_right,top_right,top_left])

    pts = np.array([bottom_left,bottom_right,top_right,top_left], np.int32)
    pts = pts.reshape((-1,1,2))
    copy = img.copy()
    cv2.polylines(copy,[pts],True,(255,0,0), thickness=3)

    # Destination points are chosen such that straight lanes appear more or less parallel in the transformed image.
    bottom_left = [320,720]
    bottom_right = [920, 720]
    top_left = [320, 1]
    top_right = [920, 1]

    dst = np.float32([bottom_left,bottom_right,top_right,top_left])
    M = cv2.getPerspectiveTransform(source, dst)
    M_inv = cv2.getPerspectiveTransform(dst, source)
    img_size = (img.shape[1], img.shape[0])

    warped = cv2.warpPerspective(img, M, img_size , flags=cv2.INTER_LINEAR)
    return warped, M_inv
        
warped_images = apply(warp, roi_images)
warped_images = [warped_image[0] for warped_image in warped_images]
# show_images(warped_images, cmap='gray')